# Cvičení 3: Bayesovská statistika – hledání místa havárie

**Motivace:** Letadlo se zřítilo do moře. Nevíme kde, ale máme informace:
- poslední známou polohu (0,0) v čase $t_0=0$,
- odhad směru letu (úhel s nejistotou),
- časové okno havárie,
- několik pozdějších nálezů trosek.

**Hlavní myšlenka (Bayes):** Spojíme
- **prior**: co si myslíme před měřením,
- **likelihood**: jak dobře data sedí na model,
- **posterior**: výsledný odhad polohy.

Vše počítáme v rovině (x,y) a v jednotkách metr/sekunda.

<img src="images/bayes_overview.png" alt="Bayes overview" width="750"/>

**Cíle:**
- napsat PDF pro uniformní a normální rozdělení,
- převést prior z $(T, \Theta)$ do roviny $(x,y)$,
- sestavit jednoduchý model měření (likelihood),
- spočítat posterior na mřížce a najít MAP odhad.


---
## 0. Příprava notebooku

Budeme používat:
- `numpy` a `matplotlib`,
- `numba` pro rychlé smyčky v úloze 7,
- `cv3_utils.py` (black-box drift z CV2 + pomocné vykreslení).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange

from cv3_utils import (
    default_params,
    make_semicircle_grid,
    make_km_axes,
    plot_pdf_pair,
    plot_heatmap_km,
    plot_trajectory_km,
    normalize_on_grid,
    mask_outside_semicircle,
    w_synthetic_point,
    simulate_drag_w_synthetic,
)


---
## 1. Náhodné veličiny: čas havárie a směr letu


Máme dvě náhodné veličiny:

1. **Čas havárie** $T$ (od posledního check-upu):
   $$T \sim \mathrm{Uniform}(t_{\min}, t_{\max}).$$

2. **Směr letu** jako úhel $\Theta$:
   $$\Theta \sim \mathcal{N}(\mu_\theta, \sigma_\theta^2).$$

Nejprve si napíšeme hustoty (PDF) těchto rozdělení a vykreslíme je.


## Úkol 1: PDF pro uniformní a normální rozdělení

**Zadání:** Implementujte `pdf_uniform` a `pdf_normal` a vykreslete je.

Použijte parametry:
- $t_{\min}=60\,\mathrm{s}$, $t_{\max}=960\,\mathrm{s}$ (okno 1–16 min),
- $\mu_\theta=\pi/2$, $\sigma_\theta=\pi/3$,
- vykreslete $T$ v minutách pro $t\in[t_{\min}-60\,\mathrm{s}, t_{\max}+60\,\mathrm{s}]$,
- vykreslete $\Theta$ na intervalu $[-\pi,\pi]$.


In [ ]:
def pdf_uniform(t, tmin, tmax):
    "Hustota Uniform(tmin, tmax)."
    t = np.asarray(t, dtype=float)
    out = np.zeros_like(t)
    mask = (t >= tmin) & (t <= tmax)
    out[mask] = 1.0 / (tmax - tmin)
    return out


def pdf_normal(x, mu, sig):
    "Hustota N(mu, sig^2)."
    x = np.asarray(x, dtype=float)
    z = (x - mu) / sig
    return np.exp(-0.5 * z * z) / (sig * np.sqrt(2 * np.pi))


params = default_params()
tmin = params["T_WINDOW_MIN"]
tmax = params["T_WINDOW_MAX"]
theta_mu = params["THETA_MU"]
theta_sig = params["THETA_SIG"]

t_plot = np.linspace(tmin - 60.0, tmax + 60.0, 400)
theta_plot = np.linspace(-np.pi, np.pi, 600)

plot_pdf_pair(
    t_plot / 60.0,
    pdf_uniform(t_plot, tmin, tmax),
    "t [min]",
    "PDF času havárie T (uniformní)",
    theta_plot,
    pdf_normal(theta_plot, theta_mu, theta_sig),
    "θ [rad]",
    "PDF směru letu Θ (normální)",
)


---
## 2. Prior v rovině (x,y)


Letadlo letí konstantní rychlostí $V$. Pokud havaruje po čase $T$ a s úhlem $\Theta$, pak
$$
x = V T \cos\Theta,\qquad y = V T \sin\Theta.
$$

Prior v rovině vznikne transformací rozdělení $(T,\Theta)$:
$$
f_U(x,y) = \frac{f_T(r/V)\,f_\Theta(\theta)}{V\,r},
\quad r=\sqrt{x^2+y^2},\ \theta=\mathrm{atan2}(y,x).
$$

Intuice: prior je „klín“ kolem směru $\mu_\theta$ a s časovým oknem $[t_{\min}, t_{\max}]$.

<img src="images/prior_wedge.png" alt="prior wedge" width="450"/>


## Úkol 2: Implementujte hustotu prioru v rovině

**Zadání:** Doplňte funkci `prior_density_xy` a vykreslete prior na mřížce.

Parametry:
- $V = 800\,\mathrm{km/h}$ (převeďte na m/s),
- $t_{\min}, t_{\max}, \mu_\theta, \sigma_\theta$ jako v Úkolu 1,
- mřížka: půlkruh s poloměrem $R = V\,t_{\max}$,
- rozlišení: `GRID_N = 160` bodů v obdélníku $[-R, R] \times [0, R]$.

Tip: Pro mřížku použijte helper `make_semicircle_grid(...)` z `cv3_utils.py`.


In [ ]:
def prior_density_xy(x, y, V, tmin, tmax, theta_mu, theta_sig):
    "Prior hustota f_U(x,y) daná uniformním časem a normálním úhlem směru."
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    r = np.sqrt(x * x + y * y)
    theta = np.arctan2(y, x)
    t = r / V

    out = np.zeros_like(r)
    mask = (r > 0.0) & (t >= tmin) & (t <= tmax)

    fT = 1.0 / (tmax - tmin)
    fTheta = pdf_normal(theta[mask], theta_mu, theta_sig)
    out[mask] = (fT * fTheta) / (V * r[mask])
    return out


params = default_params()
V = params["V_PLANE"]
tmin = params["T_WINDOW_MIN"]
tmax = params["T_WINDOW_MAX"]
theta_mu = params["THETA_MU"]
theta_sig = params["THETA_SIG"]
grid_n = params["GRID_N"]

xg, yg, XX, YY = make_semicircle_grid(V, tmax, grid_n)
prior = prior_density_xy(XX, YY, V, tmin, tmax, theta_mu, theta_sig)

radius = V * tmax
prior_plot = mask_outside_semicircle(prior, XX, YY, radius)

plot_heatmap_km(prior_plot, xg, yg, "Prior: hustota místa havárie", "f_U(x,y) [1/m²]")
plt.show()


---
## 3. Z bodu (x,y) dopočítáme čas havárie


Známe-li kandidátní bod havárie $(x,y)$, odpovídající čas je
$$
T(x,y) = \frac{\sqrt{x^2+y^2}}{V}.
$$
Tento čas potřebujeme, protože drift trosek začíná až v okamžiku havárie.


## Úkol 3: Funkce pro čas havárie z (x,y)

**Zadání:** Doplňte funkci `crash_time_from_xy` a vykreslete mapu času havárie
na stejné mřížce jako v Úkolu 2.

Parametry:
- $V = 800\,\mathrm{km/h}$,
- mřížka: půlkruh s poloměrem $R = V\,t_{\max}$ a `GRID_N = 160`.


In [ ]:
def crash_time_from_xy(x, y, V):
    "Čas havárie (od posledního check-upu) odpovídající bodu (x,y)."
    r = np.sqrt(x * x + y * y)
    return r / V


params = default_params()
V = params["V_PLANE"]
tmax = params["T_WINDOW_MAX"]
grid_n = params["GRID_N"]

xg, yg, XX, YY = make_semicircle_grid(V, tmax, grid_n)
t_crash = crash_time_from_xy(XX, YY, V)

radius = V * tmax
t_crash_plot = mask_outside_semicircle(t_crash / 3600.0, XX, YY, radius)

plot_heatmap_km(t_crash_plot, xg, yg, "Čas havárie t_crash", "t_crash [h]")
plt.show()


---
## 4. Forward model driftu trosek (black box)

Z CV2 máme model driftu v časově proměnném poli $\mathbf{w}(\mathbf{r},t)$.
V tomto cvičení ho bereme jako **black box**:

- vstup: místo havárie $u=(x,y)$, parametry trosky ($\kappa$), čas nálezu,
- výstup: předpovězená poloha nálezu.

Použijeme solver `simulate_drag_w_synthetic` z `cv3_utils.py` (krok $\Delta t=100\,\mathrm{s}$).


## Úkol 4: Otestujte solver na jednom bodě

**Zadání:** Zvolte jeden kandidátní bod havárie a vykreslete trajektorii.

Parametry:
- $V = 800\,\mathrm{km/h}$ a okno $t_{\min}, t_{\max}$,
- bod $u_0 = (0,\ 0{.}6\,V\,t_{\max})$,
- $\kappa = 2{.}0\times 10^{-2}\,\mathrm{m}^{-1}$,
- doba driftu $T = 5\,\mathrm{dní}$,
- krok $\Delta t = 100\,\mathrm{s}$,
- parametry pole: `SPATIAL_FREQ = 1.7`, `TEMPORAL_FREQ = 1.0`, `L_M = 1.2e6 m`.

Výstup: trajektorie v rovině $(x,y)$ se startem a koncem.


In [ ]:
params = default_params()
V = params["V_PLANE"]
tmax = params["T_WINDOW_MAX"]

u0 = np.array([0.0, 0.6 * V * tmax])
kappa = 2.0e-2
T_drift = 5.0 * 24.0 * 3600.0

t_crash_u0 = crash_time_from_xy(u0[0], u0[1], V)

_, r, _, _ = simulate_drag_w_synthetic(
    u0.astype(np.float64),
    float(t_crash_u0),
    float(kappa),
    float(T_drift),
    dt=float(params["DT_DRIFT"]),
    spatial_freq=float(params["SPATIAL_FREQ"]),
    temporal_freq=float(params["TEMPORAL_FREQ"]),
    L_M=float(params["L_M"]),
)

fig, ax = make_km_axes(title="Ukázka: trajektorie jedné trosky")
plot_trajectory_km(
    r,
    ax=ax,
    start_label="start (havárie)",
    end_label="konec (predikce)",
)
ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5))
fig.subplots_adjust(right=0.78)
plt.show()


---
## 5. Likelihood: model měření polohy trosky


Reálný svět je nejistý: model driftu je zjednodušený a měření polohy má chybu.
Zavedeme proto jednoduchý šum:

$$y = G(u) + z,\qquad z \sim \mathcal{N}(0, \sigma^2 I).$$

Směrodatná odchylka roste s dobou driftu:
$$\sigma(\text{days}) = \sigma_0 + c\cdot \text{days}.$$


## Úkol 5: Implementujte sigma(t) a log-likelihood

**Zadání:** Napište funkce:
- `sigma_of_days(days, sigma0_km, sigma_per_day_km)` → vrací $\sigma$ v metrech,
- `loglike_isotropic_gauss(y_obs, y_pred, sigma_m)`.

Parametry:
- $\sigma_0 = 100\,\mathrm{km}$,
- růst $10\,\mathrm{km/den}$.


In [ ]:
def sigma_of_days(days, sigma0_km, sigma_per_day_km):
    "Směrodatná odchylka měření (v metrech) jako funkce doby driftu ve dnech."
    return (sigma0_km + sigma_per_day_km * days) * 1000.0


def loglike_isotropic_gauss(y_obs, y_pred, sigma_m):
    "Log-likelihood pro izotropní 2D Gauss: N(y_obs | y_pred, sigma^2 I)."
    dx = y_obs[0] - y_pred[0]
    dy = y_obs[1] - y_pred[1]
    r2 = dx * dx + dy * dy
    return -0.5 * r2 / (sigma_m * sigma_m) - np.log(2 * np.pi * sigma_m * sigma_m)


params = default_params()
sigma0_km = params["SIGMA0_KM"]
sigma_per_day_km = params["SIGMA_PER_DAY_KM"]

for d in [0, 1, 3, 7]:
    print(f"days={d:>2}  sigma={sigma_of_days(d, sigma0_km, sigma_per_day_km)/1000:.2f} km")


---
## 6. Syntetická data: vygenerujeme „nálezy trosek"


Abychom si mohli ověřit, že Bayesovská inverze funguje, vygenerujeme umělá data:

1. Vylosujeme „pravý“ čas a směr havárie z prioru.
2. Spočteme pravý bod havárie $u_\mathrm{true}$.
3. Pro každou trosku vytvoříme pozorování: modelový drift + Gaussovský šum.

Použijeme pevný seed, aby data byla reprodukovatelná.


## Úkol 6: Vygenerujte data pro několik trosek

**Zadání:** Vygenerujte syntetická data a vykreslete je.

Parametry:
- RNG seed = 12345,
- počet trosek $N=3$,
- $\kappa$ pro tři trosky: $1{.}5,\ 2{.}5,\ 3{.}5 \times 10^{-2}\,\mathrm{m}^{-1}$,
- časy nálezů uniformně v intervalu 3–9 dní,
- stejné parametry pole jako v Úkolu 4,
- měřicí šum podle Úkolu 5.


In [ ]:
DAY = 24.0 * 3600.0


def sample_true_crash(rng, V, tmin, tmax, theta_mu, theta_sig):
    "Vylosuje (T_true, theta_true) a vrátí u_true = (x,y)."
    T_true = rng.uniform(tmin, tmax)
    theta_true = rng.normal(theta_mu, theta_sig)
    x_true = V * T_true * np.cos(theta_true)
    y_true = V * T_true * np.sin(theta_true)
    return T_true, theta_true, np.array([x_true, y_true], dtype=np.float64)


def generate_synthetic_observations(rng_seed=12345):
    params = default_params()
    V = params["V_PLANE"]
    tmin = params["T_WINDOW_MIN"]
    tmax = params["T_WINDOW_MAX"]
    theta_mu = params["THETA_MU"]
    theta_sig = params["THETA_SIG"]

    dt = params["DT_DRIFT"]
    spatial_freq = params["SPATIAL_FREQ"]
    temporal_freq = params["TEMPORAL_FREQ"]
    L_M = params["L_M"]

    sigma0_km = params["SIGMA0_KM"]
    sigma_per_day_km = params["SIGMA_PER_DAY_KM"]

    min_find_day = params["MIN_FIND_DAY"]
    max_find_day = params["MAX_FIND_DAY"]
    n_debris = params["N_DEBRIS"]

    rng = np.random.default_rng(rng_seed)

    T_true, theta_true, u_true = sample_true_crash(rng, V, tmin, tmax, theta_mu, theta_sig)

    debris_params = [
        {"name": "troska A", "kappa": 1.5e-2},
        {"name": "troska B", "kappa": 2.5e-2},
        {"name": "troska C", "kappa": 3.5e-2},
    ][:n_debris]

    find_days = rng.uniform(min_find_day, max_find_day, size=len(debris_params))
    t_find = find_days * DAY

    observations = []
    for k, params_k in enumerate(debris_params):
        kappa = float(params_k["kappa"])
        t_find_k = float(t_find[k])
        T_drift = t_find_k - float(T_true)

        _, r_sim, _, _ = simulate_drag_w_synthetic(
            u_true,
            float(T_true),
            kappa,
            float(T_drift),
            dt=float(dt),
            spatial_freq=float(spatial_freq),
            temporal_freq=float(temporal_freq),
            L_M=float(L_M),
        )
        y_pred = r_sim[-1].copy()

        sigma_k = sigma_of_days(T_drift / DAY, sigma0_km, sigma_per_day_km)
        noise = rng.normal(0.0, sigma_k, size=2)
        y_obs = y_pred + noise

        observations.append(
            {
                "name": params_k["name"],
                "kappa": kappa,
                "t_find": t_find_k,
                "T_drift": T_drift,
                "y_pred_true": y_pred,
                "y_obs": y_obs,
                "sigma": sigma_k,
                "r_path": r_sim,
            }
        )

    return u_true, T_true, theta_true, observations


u_true, T_true, theta_true, observations = generate_synthetic_observations()

fig, ax = make_km_axes(title="Syntetická data: nálezy trosek")
ax.scatter([u_true[0] / 1000.0], [u_true[1] / 1000.0], marker="*", s=180, label="pravá havárie u_true")

for obs in observations:
    plot_trajectory_km(obs["r_path"], ax=ax, show_start=False, show_end=False)
    ax.scatter([obs["y_obs"][0] / 1000.0], [obs["y_obs"][1] / 1000.0], s=60, label=f"{obs['name']} (pozor.)")

ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5))
fig.subplots_adjust(right=0.78)
plt.show()

for obs in observations:
    print(
        f"{obs['name']}: day={obs['t_find'] / DAY:.2f}  sigma={obs['sigma'] / 1000:.2f} km  y_obs[km]={obs['y_obs'] / 1000}"
    )


---
## 7. Rychlé vyhodnocení forward modelu na mřížce


Vyhodnocovat celý solver pro každý bod mřížky je pomalé.
Stačí nám **finální poloha** po době driftu, proto použijeme minimalistickou Numba funkci.


## Úkol 7: Numba funkce pro finální polohu

**Zadání:** Doplňte jádro integrace v `drift_final_single` a vyhodnoťte
forward model na celé mřížce pro první trosku.

Parametry:
- $\Delta t = 100\,\mathrm{s}$,
- parametry pole: `SPATIAL_FREQ = 1.7`, `TEMPORAL_FREQ = 1.0`, `L_M = 1.2e6 m`,
- stejná mřížka jako v Úkolu 2 (půlkruh s poloměrem $R = V\,t_{\max}$),
- syntetická data z Úkolu 6 (seed 12345).


In [ ]:
@njit
def drift_final_single(x0, y0, t0, kappa, T, dt, spatial_freq, temporal_freq, L_M):
    "Vrátí finální polohu po driftu délky T, start v (x0,y0) v čase t0."
    rx = x0
    ry = y0
    t = t0

    wx, wy = w_synthetic_point(rx, ry, t, spatial_freq, temporal_freq, L_M)
    vx = wx
    vy = wy

    n = int(np.floor(T / dt)) + 1
    for _ in range(1, n):
        wx, wy = w_synthetic_point(rx, ry, t, spatial_freq, temporal_freq, L_M)

        vrel_x = vx - wx
        vrel_y = vy - wy
        vrel_n = (vrel_x * vrel_x + vrel_y * vrel_y) ** 0.5

        ax = -kappa * vrel_n * vrel_x
        ay = -kappa * vrel_n * vrel_y

        vx_new = vx + ax * dt
        vy_new = vy + ay * dt

        rx = rx + 0.5 * (vx + vx_new) * dt
        ry = ry + 0.5 * (vy + vy_new) * dt

        vx = vx_new
        vy = vy_new
        t = t + dt
    return rx, ry


@njit(parallel=True)
def forward_grid_final(XX, YY, T_crash, t_find, kappa, dt, spatial_freq, temporal_freq, L_M):
    "Pro každý bod mřížky spočte finální polohu v čase t_find pro danou trosku."
    ny, nx = XX.shape
    outx = np.empty((ny, nx), dtype=np.float64)
    outy = np.empty((ny, nx), dtype=np.float64)

    for j in prange(ny):
        for i in range(nx):
            t0 = T_crash[j, i]
            T = t_find - t0
            fx, fy = drift_final_single(XX[j, i], YY[j, i], t0, kappa, T, dt, spatial_freq, temporal_freq, L_M)
            outx[j, i] = fx
            outy[j, i] = fy
    return outx, outy


params = default_params()
V = params["V_PLANE"]
tmax = params["T_WINDOW_MAX"]
grid_n = params["GRID_N"]

xg, yg, XX, YY = make_semicircle_grid(V, tmax, grid_n)
t_crash = crash_time_from_xy(XX, YY, V)

u_true, T_true, theta_true, observations = generate_synthetic_observations()
obs0 = observations[0]

FX0, FY0 = forward_grid_final(
    XX.astype(np.float64),
    YY.astype(np.float64),
    t_crash.astype(np.float64),
    float(obs0["t_find"]),
    float(obs0["kappa"]),
    float(params["DT_DRIFT"]),
    float(params["SPATIAL_FREQ"]),
    float(params["TEMPORAL_FREQ"]),
    float(params["L_M"]),
)

radius = V * tmax
FX0_plot = mask_outside_semicircle(FX0 / 1000.0, XX, YY, radius)
FY0_plot = mask_outside_semicircle(FY0 / 1000.0, XX, YY, radius)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
plot_heatmap_km(FX0_plot, xg, yg, "Forward G(u): x_final (troska 1)", "x_final [km]", ax=axes[0])
plot_heatmap_km(FY0_plot, xg, yg, "Forward G(u): y_final (troska 1)", "y_final [km]", ax=axes[1])
plt.tight_layout()
plt.show()


---
## 8. Likelihood a posterior na mřížce (1 troska)


Pro jednu trosku platí:
$$
f_{U|Y}(u|y) \propto f_{Y|U}(y|u)\,f_U(u).
$$

Na mřížce budeme počítat **log-posterior** a pak vše znormalizujeme.


In [ ]:
params = default_params()
V = params["V_PLANE"]
tmin = params["T_WINDOW_MIN"]
tmax = params["T_WINDOW_MAX"]
theta_mu = params["THETA_MU"]
theta_sig = params["THETA_SIG"]
grid_n = params["GRID_N"]

xg, yg, XX, YY = make_semicircle_grid(V, tmax, grid_n)
prior = prior_density_xy(XX, YY, V, tmin, tmax, theta_mu, theta_sig)
t_crash = crash_time_from_xy(XX, YY, V)

u_true, T_true, theta_true, observations = generate_synthetic_observations()
obs0 = observations[0]

FX0, FY0 = forward_grid_final(
    XX.astype(np.float64),
    YY.astype(np.float64),
    t_crash.astype(np.float64),
    float(obs0["t_find"]),
    float(obs0["kappa"]),
    float(params["DT_DRIFT"]),
    float(params["SPATIAL_FREQ"]),
    float(params["TEMPORAL_FREQ"]),
    float(params["L_M"]),
)

log_prior = np.where(prior > 0, np.log(prior), -np.inf)

y_obs0 = obs0["y_obs"].astype(np.float64)
sigma0 = float(obs0["sigma"])
DX = y_obs0[0] - FX0
DY = y_obs0[1] - FY0
R2 = DX * DX + DY * DY
log_like0 = -0.5 * R2 / (sigma0 * sigma0) - np.log(2 * np.pi * sigma0 * sigma0)

log_post0 = log_prior + log_like0
m = np.max(log_post0[np.isfinite(log_post0)])
post_unn = np.exp(log_post0 - m)
post_unn[~np.isfinite(log_post0)] = 0.0
post0, Z0 = normalize_on_grid(post_unn, xg, yg)

j_map, i_map = np.unravel_index(np.argmax(post0), post0.shape)
u_map = np.array([xg[i_map], yg[j_map]])

radius = V * tmax
prior_plot = mask_outside_semicircle(prior, XX, YY, radius)
like_rel = np.exp(log_like0 - np.max(log_like0))
like_rel_plot = mask_outside_semicircle(like_rel, XX, YY, radius)
post_plot = mask_outside_semicircle(post0, XX, YY, radius)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
plot_heatmap_km(prior_plot, xg, yg, "Prior f_U(u)", "prior [1/m²]", ax=axes[0, 0])
plot_heatmap_km(like_rel_plot, xg, yg, "Likelihood f(y|u) (relativní)", "rel. likelihood", ax=axes[0, 1])

plot_heatmap_km(post_plot, xg, yg, "Posterior po 1 trosce", "posterior [1/m²]", ax=axes[1, 0])
axes[1, 0].scatter([u_true[0] / 1000.0], [u_true[1] / 1000.0], marker="*", s=200, c="white", edgecolors="black", label="u_true")
axes[1, 0].scatter([u_map[0] / 1000.0], [u_map[1] / 1000.0], marker="x", s=120, c="white", label="MAP")
axes[1, 0].legend(loc="upper right")

axes[1, 1].axis("off")
axes[1, 1].text(0.02, 0.85, f"Normalizační konstanta Z0 ≈ {Z0:.3e}", fontsize=12)
axes[1, 1].text(0.02, 0.70, f"u_true = ({u_true[0]/1000:.2f}, {u_true[1]/1000:.2f}) km", fontsize=12)
axes[1, 1].text(0.02, 0.55, f"u_MAP  = ({u_map[0]/1000:.2f}, {u_map[1]/1000:.2f}) km", fontsize=12)
axes[1, 1].text(0.02, 0.40, f"t_true = {T_true/60:.2f} min", fontsize=12)
axes[1, 1].text(0.02, 0.25, f"θ_true = {theta_true:.2f} rad", fontsize=12)

plt.tight_layout()
plt.show()


---
## 9. Více trosek: nezávislé likelihoody a finální posterior

Předpoklad „nezávislosti trosek“ znamená:
$$
f(y^{(1)},\dots,y^{(K)}|u) = \prod_{k=1}^K f(y^{(k)}|u).
$$

V logaritmech tedy sčítáme log-likelihoody.


## Úkol 9: Posterior pro více trosek

**Zadání:** Implementujte postupné aktualizace posterioru pro seznam trosek:
- pro každou trosku vykreslete její likelihood (alespoň relativně),
- nakonec vykreslete finální posterior a MAP odhad.


In [ ]:
def compute_log_posterior_all(observations, XX, YY, t_crash, log_prior, dt, spatial_freq, temporal_freq, L_M):
    "Vrátí log-posterior (nenormovaný) po všech troskách."
    log_post = log_prior.copy()
    like_images = []

    for obs in observations:
        kappa = float(obs["kappa"])
        t_find = float(obs["t_find"])
        y_obs = obs["y_obs"].astype(np.float64)
        sigma = float(obs["sigma"])

        FX, FY = forward_grid_final(XX, YY, t_crash, t_find, kappa, dt, spatial_freq, temporal_freq, L_M)

        DX = y_obs[0] - FX
        DY = y_obs[1] - FY
        R2 = DX * DX + DY * DY
        log_like = -0.5 * R2 / (sigma * sigma) - np.log(2 * np.pi * sigma * sigma)

        log_post = log_post + log_like
        like_images.append(log_like)

    return log_post, like_images


params = default_params()
V = params["V_PLANE"]
tmin = params["T_WINDOW_MIN"]
tmax = params["T_WINDOW_MAX"]
theta_mu = params["THETA_MU"]
theta_sig = params["THETA_SIG"]
grid_n = params["GRID_N"]

xg, yg, XX, YY = make_semicircle_grid(V, tmax, grid_n)
prior = prior_density_xy(XX, YY, V, tmin, tmax, theta_mu, theta_sig)
t_crash = crash_time_from_xy(XX, YY, V)
log_prior = np.where(prior > 0, np.log(prior), -np.inf)

u_true, T_true, theta_true, observations = generate_synthetic_observations()

log_post_all, log_likes = compute_log_posterior_all(
    observations,
    XX.astype(np.float64),
    YY.astype(np.float64),
    t_crash.astype(np.float64),
    log_prior.astype(np.float64),
    float(params["DT_DRIFT"]),
    float(params["SPATIAL_FREQ"]),
    float(params["TEMPORAL_FREQ"]),
    float(params["L_M"]),
)

m = np.max(log_post_all[np.isfinite(log_post_all)])
post_unn = np.exp(log_post_all - m)
post_unn[~np.isfinite(log_post_all)] = 0.0
post_all, Z = normalize_on_grid(post_unn, xg, yg)

radius = V * tmax
prior_plot = mask_outside_semicircle(prior, XX, YY, radius)
post_plot = mask_outside_semicircle(post_all, XX, YY, radius)

fig = plt.figure(figsize=(12, 3.6 + 3.2))

n = len(observations)
for k in range(n):
    ax = plt.subplot(2, max(n, 2), k + 1)
    Lk = log_likes[k]
    like_rel = np.exp(Lk - np.max(Lk))
    like_plot = mask_outside_semicircle(like_rel, XX, YY, radius)
    plot_heatmap_km(
        like_plot,
        xg,
        yg,
        f"Likelihood: {observations[k]['name']}",
        "rel. likelihood",
        ax=ax,
        show_colorbar=False,
    )

ax_post = plt.subplot(2, 1, 2)
plot_heatmap_km(post_plot, xg, yg, "Finální posterior po všech troskách", "posterior [1/m²]", ax=ax_post)

j_map, i_map = np.unravel_index(np.argmax(post_all), post_all.shape)
u_map = np.array([xg[i_map], yg[j_map]])

ax_post.scatter([u_true[0] / 1000.0], [u_true[1] / 1000.0], marker="*", s=220, c="white", edgecolors="black", label="u_true")
ax_post.scatter([u_map[0] / 1000.0], [u_map[1] / 1000.0], marker="x", s=150, c="white", label="MAP")
ax_post.legend(loc="upper right")

plt.tight_layout()
plt.show()

print(f"u_true [km] = ({u_true[0] / 1000.0:.2f}, {u_true[1] / 1000.0:.2f})")
print(f"u_MAP  [km] = ({u_map[0] / 1000.0:.2f}, {u_map[1] / 1000.0:.2f})")


---

## Poznámky k interpretaci a rozšíření (volitelné)

1. **Co ovlivňuje šířku posterioru**
   - širší prior (větší $\sigma_\theta$) → větší neurčitost,
   - větší šum ($\sigma_0$, $c$) → menší informativnost trosek,
   - více trosek → typicky užší posterior.

2. **Proč logaritmy**
   - součin mnoha malých likelihoodů může numericky podtéct k nule,
   - v logaritmech se součiny mění na součty.

3. **Omezení modelu**
   - drift model (CV2) je zjednodušený,
   - šum v poloze je hrubá aproximace mnoha zdrojů nejistot,
   - nezávislost trosek je idealizace.
